In [20]:
import os
import numpy as np
from skimage import io, measure
from skimage.measure import regionprops
import glob
from ultralytics import YOLO

import torch
import napari
import skimage as ski
import pandas as pd
import plotly.express as px

In [2]:
viewer = napari.Viewer()

In [3]:
variant = '36'

In [4]:
model = YOLO('runs/detect/train'+variant+'/weights/best.pt')  # load a custom model

In [5]:
def pad_to_window(img, window=10000):
    height, width = img.shape[0:2]
    new_img = np.pad(img, ((0, window-height), (0, window-width), (0,0)), mode='constant', constant_values=255)
    return new_img
def window_images(img, window=10000):
    shape = img.shape[0:2]
    y_comb = []
    for y in range(0, shape[0], window):
        x_comb = []
        for x in range(0, shape[1], window):
            x_comb.append(pad_to_window(img[y:y+window, x:x+window]))
        y_comb.append(x_comb)
    return np.array(y_comb)

def de_window(img):
    rtn_img = np.concatenate([np.concatenate([x for x in simg], axis=1) for simg in img], axis=0)
    return rtn_img
def process_file(fname, display=False, viewer=None, confidence_cutoff=0.5):
    img = ski.io.imread(fname)
    windowed = window_images(img)
    confs = []
    rtn_boxes = []
    classes = []
    for y,yimg in enumerate(windowed):
        for x,ximg in enumerate(yimg):
            height = 10000
            width = 10000
            results = model(ximg)
            for box in results[0].boxes:
                xyxy = np.reshape(box.xyxy[0].cpu().numpy(), [-1,2])[:,::-1]
                xyxy = xyxy + np.array([y*10000, x*10000])[np.newaxis,:]
                rtn_boxes.append(xyxy)
                classes.append(box.cls.cpu().numpy()[0])
                confs.append(box.conf[0].cpu().numpy())
    classes = np.array(classes)
    rtn_boxes = np.array(rtn_boxes)
    confs = np.array(confs)

    classes = classes[confs > confidence_cutoff]
    rtn_boxes = rtn_boxes[confs > confidence_cutoff]
    confs = confs[confs > confidence_cutoff]
    rtn_img = de_window(windowed)

    if display:
        viewer.layers.clear()
        viewer.add_image(rtn_img, channel_axis=-1, colormap=['red', 'green', 'blue'])
        viewer.add_shapes(rtn_boxes[classes==0], shape_type='rectangle', edge_color='red', face_color='red', opacity=0.5, name='T0')
        viewer.add_shapes(rtn_boxes[classes==1], shape_type='rectangle', edge_color='blue', face_color='blue', opacity=0.5, name='T1')
        viewer.add_shapes(rtn_boxes[classes==2], shape_type='rectangle', edge_color='green', face_color='green', opacity=0.5, name='T2')
        viewer.add_shapes(rtn_boxes[classes==3], shape_type='rectangle', edge_color='yellow', face_color='yellow', opacity=0.5, name='T3')

    return rtn_img, rtn_boxes, classes, confs



In [6]:
df = pd.read_csv('Counts_'+variant+'.csv')
df['class'] = df['class'].astype(str)

In [7]:
df['Tray'] = df['file'].str.split('\\').str[-1].str.split('_').str[0].astype(int)
df['Slide'] = df['file'].str.split('\\').str[-1].str.split('_').str[1].astype(int)
df['Object'] = df['file'].str.split('\\').str[-1].str.split('_').str[2].str[0:2].astype(int)

In [8]:
surface_mask = ((df['Tray']==1) & (df['Slide']<21)) | ((df['Tray']==2) & (df['Slide']>10) & (df['Slide']<16))
pachon_mask = ((df['Tray']==1) & (df['Slide']>20) & (df['Slide']<41)) | ((df['Tray']==2) & (df['Slide']>15) & (df['Slide']<21))
molino_mask = ((df['Tray']==1) & (df['Slide']>39) & (df['Slide']<51)) | ((df['Tray']==2) & (df['Slide']>0) & (df['Slide']<11)) | ((df['Tray']==2) & (df['Slide']>19) & (df['Slide']<26))

In [9]:
df['Genotype'] = 'None'
df.loc[surface_mask, 'Genotype'] = 'Surface'
df.loc[pachon_mask, 'Genotype'] = 'Pachon'
df.loc[molino_mask, 'Genotype'] = 'Molino'

In [22]:
df.to_csv('S:/micro/nro/fx2482/lem/smc/csv_files/20240613.csv')

In [15]:
agged = df.groupby(['Genotype', 'class']).agg({'conf': 'count', 'area':'median'}).reset_index().rename(columns={'conf': 'Count'})
agged_sum = agged.groupby('Genotype').agg({'Count': 'sum'}).reset_index().rename(columns={'Count': 'Total'})
agged = pd.merge(agged, agged_sum, on='Genotype')
agged['Fraction'] = agged['Count'] / agged['Total']

In [16]:
f = px.bar(agged, x='Genotype', y='Fraction', color='class', color_discrete_map=color_map, category_orders=category_orders, width=600)
f.write_html('Fractions_'+variant+'.html')
f.write_image('Fractions_'+variant+'.png')
f

In [17]:
f = px.box(df, x='Genotype', y='area', facet_col='class', color='class', color_discrete_map=color_map, category_orders=category_orders, width=800)
f.write_html('Areas_'+variant+'.html')
f.write_image('Areas_'+variant+'.png')
f

In [18]:
agged = df.groupby(['file', 'class', 'Genotype']).agg({'conf': 'count'}).reset_index()

In [19]:
import plotly.graph_objects as go

agged['f'] = agged['file'].str.split('\\').str[-1]

color_map = {
    '0.0': 'red',
    '1.0': 'blue',
    '3.0': 'yellow',
    # Add more classes and colors as needed
}

category_orders = {'class':['0.0', '1.0', '3.0'], 'Genotype':['Surface', 'Pachon', 'Molino']}

f=go.FigureWidget(
    px.bar(agged, x='f', y='conf', color='class', height=800, hover_data=['file'], color_discrete_map=color_map, category_orders=category_orders, width=1600, facet_col='Genotype')
    )

def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        idx = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(idx)
        process_file(idx, display=True, viewer=viewer, confidence_cutoff=0.05)

for a in f.data:
    a.on_click(click_fn)
f.write_html('Counts_'+variant+'.html')
f.write_image('Counts_'+variant+'.png')
f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'customdata': array([['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_01_02.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_01_03.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_01_04.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_01_05.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_01_06.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_02_02.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_02_03.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\\01_02_04.tif'],
                                   ['S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMA

S:/micro/nro/fx2482/lem/smc/20240613_OSS_IMARE-128918\01_27_03.tif

0: 1600x1600 259 1_stage_IBs, 8 2_stage_IIs, 33 4_stage_late_IIIs, 38.5ms
Speed: 22.5ms preprocess, 38.5ms inference, 2.1ms postprocess per image at shape (1, 3, 1600, 1600)

0: 1600x1600 (no detections), 33.0ms
Speed: 26.8ms preprocess, 33.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1600, 1600)


In [17]:
df.shape

(56175, 9)